<a href="https://colab.research.google.com/github/imabari/rakuten_tools/blob/main/rakuten_mymap_near.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# 実行方法

1. 上の「Open in Colab」をクリック（Googleアカウント必要）
2. 設定にマイマップのmidを貼り付け
3. 上部メニューより「ランタイム」を選択
4. 「すべてのセルを実行」を選択
5. アップロードのところで停止するので参照を押してTowerCollectorのファイルを指定

# 設定

In [ ]:
# マイマップのmid
mid = "152eagLReFE8oDs_thOuVxxXSx8IhegsS"
lid = "TPK-JtNO-wk"

# アップロード

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))

In [ ]:
!pip install geopandas

In [ ]:
import zipfile

In [ ]:
import requests
import pandas as pd
import geopandas as gpd
from lxml import etree

# マイマップ

In [ ]:
url = f"https://www.google.com/maps/d/kml?mid={mid}&lid={lid}"

In [ ]:
!wget "$url" -O rakuten.kmz

In [ ]:
with zipfile.ZipFile("rakuten.kmz") as existing_zip:
    existing_zip.extractall("kmz")

In [ ]:
ns = {"kml": "http://www.opengis.net/kml/2.2"}
tree = etree.parse("./kmz/doc.kml")

In [ ]:
data = []

for i in tree.xpath("//kml:Placemark", namespaces=ns):

    d = {}

    d["場所"] = "".join(i.xpath("./kml:name/text()", namespaces=ns)).strip()
    d["geometry"] = "".join(
        i.xpath("./kml:Point/kml:coordinates/text()", namespaces=ns)
    ).strip()

    for j in i.xpath("./kml:ExtendedData/kml:Data", namespaces=ns):

        k = j.attrib["name"].strip()
        v = "".join(j.xpath("./kml:value/text()", namespaces=ns)).strip()

        d[k] = v

    data.append(d)

In [ ]:
df_map = pd.DataFrame(data)

In [ ]:
df_map[["経度", "緯度", "標高"]] = df_map["geometry"].str.split(",", expand=True)

In [ ]:
# 文字から数値に変換
df_map["経度"] = pd.to_numeric(df_map["経度"])
df_map["経度"] = pd.to_numeric(df_map["経度"])

In [ ]:
# 緯度・経度が欠損の場合は削除
df_map.dropna(subset=["緯度", "経度"], how="any", inplace=True)
df_map

# プログラム

In [ ]:
import pandas as pd

In [ ]:
df0 = pd.read_csv(fn, parse_dates=["measured_at"]).dropna(how="all", axis=1).dropna(how="any", subset=["cell_id", "psc", "ta"]).astype({"cell_id": int, "short_cell_id": int, "rnc": int, "ta": int})
df0

In [ ]:
# 型確認
df0.dtypes

In [ ]:
# 日時変換
df0["measured_at"] = df0["measured_at"].dt.tz_convert("Asia/Tokyo").dt.tz_localize(None)

In [ ]:
# バンド指定
df1 = df0.query("mcc == 440 & mnc == 11").copy()
df1

In [ ]:
df1["id"] = df1["short_cell_id"].astype(str) + "-" + df1["rnc"].astype(str)

# PCI確認

In [ ]:
df_pci = df1.drop_duplicates(subset=["cell_id"]).pivot(index="short_cell_id", columns="rnc", values="psc").astype("Int64")
df_pci.fillna(0).astype(str).replace("0", "-")

In [ ]:
df_pci.to_csv("pci.csv", encoding="utf_8_sig")

# 最良値を抽出

In [ ]:
df2 = df1.groupby(["cell_id"])["dbm"].max().reset_index()
df2

In [ ]:
df3 = pd.merge(df1, df2)
df3

In [ ]:
import geopandas as gpd
from shapely.geometry import Point

In [ ]:
# 全部
# geo_df = gpd.GeoDataFrame(df1, geometry=gpd.points_from_xy(df1.lon, df1.lat), crs=2446)

In [ ]:
# 最良値のみ
geo_df = gpd.GeoDataFrame(df3, geometry=gpd.points_from_xy(df3.lon, df3.lat), crs=2446)

In [ ]:
geo_df

In [ ]:
dfs = []

for i, r in df_map.iterrows():
    
    point = Point(r["経度"], r["緯度"])

    # 300mの範囲
    buffer = point.buffer(0.003)

    df_tmp = geo_df[geo_df.geometry.within(buffer)].copy()

    if len(df_tmp) > 0:

        df_tmp["grp"] = i

        df_tmp["場所"] = r["場所"]

        df_tmp["距離"] = df_tmp.geometry.apply(lambda x: round(x.distance(point) * 10000, 3))

        dfs.append(df_tmp)


In [ ]:
df4 = pd.concat(dfs).sort_values(by=["cell_id", "距離"]).reset_index(drop=True)
df4

In [ ]:
# 重複除去
df5 = df4.drop_duplicates(subset="cell_id").copy()

In [ ]:
# 2つ以上を抽出、重複除去
# df5 = df4.groupby(by="cell_id").filter(lambda x: len(x) > 1).drop_duplicates(subset="cell_id").copy()

In [ ]:
df5

# PCI確認

In [ ]:
df5.pivot(index=["grp", "場所", "short_cell_id"], columns=["rnc"], values="psc").sort_index(level=2).fillna(0).astype(int).astype(str).replace("0", "-")

# TA確認

In [ ]:
df5.pivot(index=["grp", "場所", "short_cell_id"], columns="rnc", values="ta").sort_index(level=2).fillna(999).astype(int).astype(str).replace("999", "-")

# 距離確認

In [ ]:
df5.pivot(index=["grp", "場所", "short_cell_id"], columns=["rnc"], values="距離").sort_index(level=2).fillna(0).astype(str).replace("0", "-")